In [13]:
import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')
from hparams import create_hparams
from model import Tacotron2
import torch
from loss_function import Tacotron2Loss
from train import prepare_dataloaders
import waveglow.glow
from waveglow.denoiser import Denoiser
import IPython.display as ipd
import sys
from text import cmudict, text_to_sequence
from data_utils import TextMelLoader, TextMelCollate
import waveglow
import librosa
from layers import TacotronSTFT
from train import warm_start_model
from torch.utils.data import DataLoader
from train import save_checkpoint, load_checkpoint

sys.path.append('waveglow/')
def load_mel(path):
    audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
    audio = torch.from_numpy(audio)
    if sampling_rate != hparams.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = melspec.cuda()
    return melspec


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model

arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow).cuda().eval()
#from glow import infer

In [14]:
hparams = create_hparams(None)
hparams.n_speakers = 3
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'
hparams.batch_size = 16
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
learning_rate = 1e-3#

stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)

criterion = Tacotron2Loss()
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
iteration = 0
epoch_offset = 0
is_overflow = False

#i, batch = next(enumerate(train_loader))


torch.manual_seed(hparams.seed)
model = load_model(hparams)


In [15]:
print(hparams)

epochs=1000,iters_per_checkpoint=500,seed=1234,dynamic_loss_scaling=True,fp16_run=False,distributed_run=False,dist_backend=nccl,dist_url=tcp://localhost:54321,cudnn_enabled=True,cudnn_benchmark=False,ignore_layers=['speaker_embedding.weight'],training_files=/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt,validation_files=/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt,text_cleaners=['english_cleaners'],p_arpabet=1.0,cmudict_path=data/cmu_dictionary,max_wav_value=32768.0,sampling_rate=22050,filter_length=1024,hop_length=256,win_length=1024,n_mel_channels=80,mel_fmin=0.0,mel_fmax=8000.0,f0_min=80,f0_max=880,harm_thresh=0.25,n_symbols=185,symbols_embedding_dim=512,encoder_kernel_size=5,encoder_n_convolutions=3,encoder_embedding_dim=512,n_frames_per_step=1,decoder_rnn_dim=1024,prenet_dim=256,prenet_f0_n_layers=1,prenet_f0_dim=1,prenet_f0_kernel_size=1,prenet_rms_dim=0,prenet_rms_kernel_size=1,max_decoder_steps=

In [10]:
# hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_train.txt'
# hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_val.txt'
#hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt'
#hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'
#hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/em_train.txt'
#hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/em_val.txt'




In [11]:
#train
model = warm_start_model(
    hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'


In [20]:
import time
import datetime
#print(datetime.datetime.time())

In [21]:
timestart = time.time()
for epoch in range(5):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend

/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here
0 0 -8.44779109954834
here
0 1 -4.57365870475769
here
0 2 -3.509509563446045
here
0 3 -2.913125514984131
here
0 4 -4.987366199493408
here
0 5 -4.549313545227051
here
0 6 -3.655848264694214
here
0 7 -4.7760255336761475
here
0 8 -3.4318580627441406
here
0 9 -4.584296226501465
here
0 10 -3.4660351276397705
here
0 11 -3.8420138359069824
here
0 12 -5.872439384460449
here
0 13 -3.7943592071533203
here
0 14 -3.9080960750579834
here
0 15 -4.788084983825684
here


KeyboardInterrupt: 

In [15]:
a = next(enumerate(valset))
b = next(enumerate(valset))

In [16]:
#eval
checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uber3_5iter'
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16
model, optimizer, _learning_rate, iteration = load_checkpoint(
                checkpoint_path, model, optimizer)


Loading checkpoint '/mnt/disks/uberduck-experiments-v0/uber3_5iter'
Loaded checkpoint '/mnt/disks/uberduck-experiments-v0/uber3_5iter' from iteration 5


In [17]:
torch.cuda.empty_cache()

In [12]:
    model.eval()
    with torch.no_grad():
        val_sampler = None
        batch_size = 4
        val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                                shuffle=False, batch_size=batch_size,
                                pin_memory=False, 
                                collate_fn=collate_fn)

        val_loss = 0.0
        i, batch = next(enumerate(val_loader))
        x, y = model.parse_batch(batch)
        y_pred = model(x)
        loss = criterion(y_pred, y)

/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here


/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here


In [33]:
#!{sys.executable} -m pip install einops

In [34]:
#from einops import rearrange

In [51]:
#rearrange('i j -' x[0]
# inputs, input_lengths, targets, max_len, \
#             output_lengths, speaker_ids, f0s
x[5]

tensor([1, 1, 2, 1], device='cuda:0')

In [6]:
x5= torch.tensor([0,1,1,1],device = 'cuda:0')
xnew = (x[0][[1,2,3,0]],x[1],x[2][[1,2,3,0]],x[3],x[4],x5,x[6])

In [7]:
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.forward(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

here


'/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'

In [94]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.validation_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
#text = "its like this and like that and like this and uh"
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)
#print(audio_path, text)

# load source data to obtain rhythm using tacotron 2 as a forced aligner


In [87]:
style = torch.ones([1, 4, 4, 80]).cuda()

In [88]:
#x5= torch.tensor([0,1,1,1],device = 'cuda:0')
#text, style_input, speaker_ids, f0s 
#text_encoded.cuda(), mel.cuda(), speaker_id, pitch_contour.cuda(), rhythm[:,:].cuda()
xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

Warning! Reached max decoder steps


In [89]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [109]:
with torch.no_grad():
    # get rhythm (alignment map) using tacotron 2
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.forward(x)
    rhythm = rhythm.permute(1, 0, 2)

here


In [ ]:
(torch.Size([1, 51]),
 torch.Size([1, 80, 345]),
 tensor([1], device='cuda:0'),
 torch.Size([1, 1, 345]),
 torch.Size([345, 1, 51]))

In [111]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.validation_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
#text = "its like this and like that and like this and uh"
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)
#print(audio_path, text)


In [140]:
stft_taco = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)
path = audio_path
audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
audio = torch.from_numpy(audio)
if sampling_rate != hparams.sampling_rate:
    raise ValueError("{} SR doesn't match target {} SR".format(
        sampling_rate, stft.sampling_rate))

audio_norm = audio.unsqueeze(0)
audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)

melspec = stft_taco.mel_spectrogram(audio_norm)
melspec = melspec
mel = melspec
x, y = model.parse_batch(datacollate([dataloader[0]]))

# load source data to obtain rhythm using tacotron 2 as a forced aligner
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.forward(x)
    rhythm = rhythm.permute(1, 0, 2)

In [141]:
text_encoded.shape, mel.shape, speaker_id, pitch_contour.shape, rhythm[:,:].shape

(torch.Size([1, 59]),
 torch.Size([1, 80, 225]),
 tensor([2], device='cuda:0'),
 torch.Size([1, 1, 225]),
 torch.Size([225, 1, 59]))

In [144]:
#speaker_id = next(female_speakers) if np.random.randint(2) else next(male_speakers)
speaker_id = torch.LongTensor([2]).cuda()

with torch.no_grad():
    mel_outputs, mel_outputs_postnet, gate_outputs, _ = model.inference_noattention(
        (text_encoded.cuda(), mel.cuda(), speaker_id, pitch_contour.cuda(), rhythm[:,:].cuda()))

In [145]:
audio = denoiser(waveglow.infer(mel_outputs, sigma=0.8), 0.01)[:, 0]


In [146]:
ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate)

In [25]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [120]:
ipd.Audio(audio[1].data.cpu().numpy(), rate=hparams.sampling_rate)

In [145]:
ipd.Audio(audio[2].data.cpu().numpy(), rate=hparams.sampling_rate)

In [100]:
ipd.Audio(audio[3].data.cpu().numpy(), rate=hparams.sampling_rate)

In [36]:
save_checkpoint(model, optimizer, learning_rate, 5,
                '/mnt/disks/uberduck-experiments-v0/uber3_5iter')

Saving model and optimizer state at iteration 5 to /mnt/disks/uberduck-experiments-v0/uber3_5iter


In [55]:
print(x[0].shape,x[1],x[2].shape, x[3], x[4], x[5],x[6].shape)

torch.Size([1, 59]) tensor([59], device='cuda:0') torch.Size([1, 80, 225]) 59 tensor([225], device='cuda:0') tensor([1], device='cuda:0') torch.Size([1, 1, 225])


In [58]:
print(x[0].shape,x[1],x[2].shape, x[3], x[4], x[5],x[6].shape)

torch.Size([2, 79]) tensor([79, 59], device='cuda:0') torch.Size([2, 80, 436]) 79 tensor([436, 225], device='cuda:0') tensor([2, 1], device='cuda:0') torch.Size([2, 1, 436])


In [51]:
jayid = torch.tensor([2, 1], device='cuda:0')
benid = torch.tensor([1], device = 'cuda:0')

In [53]:

with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.forward(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs, sigma=0.8), 0.01)[:, 0]


In [75]:
mel_outputs.shape

torch.Size([5, 80, 532])

In [ ]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
    def inference(self, inputs):
        text, style_input, speaker_ids, f0s = inputs
        embedded_inputs = self.embedding(text).transpose(1, 2)
        embedded_text = self.encoder.inference(embedded_inputs)
        embedded_speakers = self.speaker_embedding(speaker_ids)[:, None]
        if hasattr(self, 'gst'):
            if isinstance(style_input, int):
                query = torch.zeros(1, 1, self.gst.encoder.ref_enc_gru_size).cuda()
                GST = torch.tanh(self.gst.stl.embed)
                key = GST[style_input].unsqueeze(0).expand(1, -1, -1)
                embedded_gst = self.gst.stl.attention(query, key)
            else:
                embedded_gst = self.gst(style_input)

        embedded_speakers = embedded_speakers.repeat(1, embedded_text.size(1), 1)
        if hasattr(self, 'gst'):
            embedded_gst = embedded_gst.repeat(1, embedded_text.size(1), 1)
            encoder_outputs = torch.cat(
                (embedded_text, embedded_gst, embedded_speakers), dim=2)
        else:
            encoder_outputs = torch.cat(
                (embedded_text, embedded_speakers), dim=2)

        mel_outputs, gate_outputs, alignments = self.decoder.inference(
            encoder_outputs, f0s)

        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments])